# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
html_page = requests.get('http://books.toscrape.com/')

In [3]:
soup = BeautifulSoup(html_page.content, 'html.parser')
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" t

In [22]:
soup.findAll('h3')[0].find('a').attrs['title']


'A Light in the Attic'

In [30]:
titles = [x.find('a').attrs['title'] for x in soup.findAll('h3')]
titles

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

In [31]:
def retrieve_titles(soup):
    titles = [x.find('a').attrs['title'] for x in soup.findAll('h3')]
    return titles



## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [32]:
import re

In [40]:
soup.findAll('p', {"class": re.compile('star-rating (.*)')})[0].attrs['class'][-1]

'Three'

In [47]:
re_cond = re.compile('star-rating (.*)')
ratings = [x.attrs['class'][-1] for x in soup.findAll('p', {"class": re_cond})]
ratings
rating_dict = {'One':1 , 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
num_ratings = [rating_dict[x] for x in ratings]
num_ratings

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

In [65]:
def retrieve_ratings(soup):
    rating_dict = {'One':1 , 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    re_cond = re.compile('star-rating (.*)')
    ratings = [x.attrs['class'][-1] for x in soup.findAll('p', {"class": re_cond})]
    num_ratings = [rating_dict[x] for x in ratings]
    return num_ratings
    #Your code here

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [52]:
soup.findAll('p', {"class": 'price_color'})[0].text[1::]

'51.77'

In [55]:
prices = [float(x.text[1::]) for x in soup.findAll('p', {"class": 'price_color'})]
prices

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

In [53]:
def retrieve_prices(soup):
    prices = [float(x.text[1::]) for x in soup.findAll('p', {"class": 'price_color'})]
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [59]:
soup.findAll('p', {"class": 'instock availability'})[0].text.strip()

'In stock'

In [62]:
availability = [x.text.strip() for x in soup.findAll('p', {"class": 'instock availability'})]
availability

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

In [60]:
def retrieve_availabilities(soup):
    availability = [x.text.strip() for x in soup.findAll('p', {"class": 'instock availability'})]
    return availability

In [77]:
import pandas as pd
cols = ['Title', 'Rating', 'Price', 'Availability']
titles = retrieve_titles(soup)
ratings = retrieve_ratings(soup)
prices = retrieve_prices(soup)
availabilities = retrieve_availabilities(soup)
df = pd.DataFrame([titles, ratings, prices, availabilities],).transpose()
df.columns = cols
display(df)

,Title,Rating,Price,Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [84]:
#Your code here
main_address = 'http://books.toscrape.com/'
html_page = requests.get(main_address)
soup = BeautifulSoup(html_page.content, 'html.parser')
soup.prettify

#Finds the maximum number
max_page = int(soup.find('li', {'class': 'current'}).text.strip().split()[-1])


In [91]:
soup.find('li', {'class': 'next'}).find('a').attrs['href']


'catalogue/page-2.html'

In [101]:
#Your code here
main_address = 'http://books.toscrape.com/'
html_page = requests.get(main_address)
soup = BeautifulSoup(html_page.content, 'html.parser')

#Finds the maximum number
max_page = int(soup.find('li', {'class': 'current'}).text.strip().split()[-1])

# initialize
titles = []
ratings = []
prices = []
availbilities = []

for page in list(range(1,max_page+1)):
    page = main_address + f'catalogue/page-{page}.html'

    html_page = requests.get(page)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
    titles += retrieve_titles(soup)
    ratings += retrieve_ratings(soup)
    prices += retrieve_prices(soup)
    availabilities += retrieve_availabilities(soup)



1000

In [109]:
import pandas as pd
cols = ['Title', 'Rating', 'Price', 'Availability']
df = pd.DataFrame([titles, ratings, prices, availabilities],).transpose()
df.columns = cols
df.dropna(inplace=True)
display(df.tail())

,Title,Rating,Price,Availability
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock
999,"1,000 Places to See Before You Die",5,26.08,In stock


In [110]:
len(df)

1000

## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [119]:
#Your code here
#Your code here
main_address = 'http://books.toscrape.com/'
html_page = requests.get(main_address)
soup = BeautifulSoup(html_page.content, 'html.parser')

#Finds the maximum number
max_page = int(soup.find('li', {'class': 'current'}).text.strip().split()[-1])

# initialize
titles = []
ratings = []
prices = []
availbilities = []

for k in list(range(1,max_page+1)):
    if k == 1:
        page = main_address + 'catalogue/page-1.html'
    else:
        page = main_address + 'catalogue/' + soup.find('li', {'class': 'next'}).find('a').attrs['href']
        
    html_page = requests.get(page)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
#     print(k, page)
    titles += retrieve_titles(soup)
    ratings += retrieve_ratings(soup)
    prices += retrieve_prices(soup)
    availabilities += retrieve_availabilities(soup)




In [120]:
import pandas as pd
cols = ['Title', 'Rating', 'Price', 'Availability']
df = pd.DataFrame([titles, ratings, prices, availabilities],).transpose()
df.columns = cols
df.dropna(inplace=True)
df

,Title,Rating,Price,Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!